# 오차역전파법  
CS231 들을 때 왜 forwardpass를 하고 또 bp(backwardpass or packpropagation)을 해서 가중치들을 왜 굳이 구할까?   
생각했었는데 앞에서 배웠던 수치 미분으로 가중치의 기울기를 구하게 되면 '너무 느려서'였다..  
별거 아니지만 나에게는 엄청난 지식을 습득한 기분!
ㄱㅂㅈㄱㅂㅈ

오차역전파법 방법에는 수식, 계산 그래프 이렇게 두 개가 있다.

## 여기선 계산 그래프를 중점으로  
이유: 중간 계산 결과를 보관할 수 있고, 역전파 시에 미분을 효율적으로 계산할 수 있기 때문

국소적 계산: 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있음

읽다보면서 든 생각이 BP하는 이유가 해당 입력이 얼마나, 어떤 영향을 미치는지 보려는거 같음

### 연쇄법칙 chain rule

역전파 계선 자철: 노드로 들어온 입력 신호에 그 노드의 국소적 미분(편미분)을 곱한 후 다음 노드로 전달

### 역전파

덧셈 노드의 역전파는 전달만  
곱셈 노드의 역전파는 순방향 입력 신호의 값이 필요해서 순전판의 입력 신호를 변수에 저장해둠

## 단순한 계층 구현하기

In [5]:
class MulLayer:
    def __init__(self):#인스턴스 변수 x, y를 순전파 시의 입력 값을 유지하기 위해 초기화
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        return out
    
    def backward(self, dout):#순전파 값을 뒤바꿔 곱한 후 출력
        dx = dout*self.y
        dy = dout*self.x
        
        return dx, dy

In [6]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [8]:
#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


여기서 backward()가 받는 인수는 순전파의 출력에 대한 미분임.

In [9]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x+y
        return out
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

In [10]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650
